## Titel: Import data
# Author: Achraf Aboukinana

# Beschreibung des Codes

Dieser Code dient der Analyse, Bereinigung und Kombination von Datensätzen, um eine zentrale und bereinigte Datenbasis zu erstellen. Die Hauptaufgaben sind wie folgt:

---

## **1. Datenimport**
- Die drei Datensätze `kiwo.csv`, `umsatzdaten_gekuerzt.csv` und `wetterdaten.csv` werden eingelesen.
- Die Spalte `Datum` wird als Datumsformat (`datetime`) geparst.

---

## **2. Datenexploration**
Eine Funktion `explore_data` analysiert jeden DataFrame:
- Form des DataFrames (Zeilen und Spalten).
- Vorschau der ersten fünf Zeilen.
- Spaltennamen, Datentypen und fehlende Werte pro Spalte.
- Beschreibende Statistiken.
- Anzahl der Duplikate.
- Zeitraum der Daten (falls `Datum`-Spalte vorhanden).

Diese Funktion wird auf alle Datensätze angewendet.

---

## **3. Zusammenführung der Datensätze**
- **Erster Merge**: `umsatzdaten_gekuerzt_data` und `wetter_data` werden anhand der Spalte `Datum` zusammengeführt.
- **Zweiter Merge**: Die zusammengeführten Daten werden mit `kiwo_data` kombiniert, wodurch eine neue Spalte `KielerWoche` entsteht:
  - Fehlende Werte in `KielerWoche` (für nicht-Kieler-Woche-Tage) werden mit `0` gefüllt.

---

## **4. Analyse fehlender Werte**
- Anzahl der fehlenden Werte (`NaN`) pro Spalte wird berechnet.
- Zeilen mit fehlenden Werten werden entfernt.

---

## **5. Speichern der Ergebnisse**
- Der kombinierte Datensatz wird in zwei Versionen gespeichert:
  - **Mit NaN-Werten**: `final_data_withNaN.csv`.
  - **Ohne NaN-Werte**: `final_data.csv`.

---

## **6. Vergleich der Zeilenanzahl**
- Die Anzahl der Zeilen vor und nach der Bereinigung wird verglichen, um den Verlust durch das Entfernen von `NaN`-Werten zu zeigen.

---

## **Zweck des Codes**
1. **Datenbereinigung**: Sicherstellung von konsistenten und vollständigen Daten.
2. **Kombination der Datensätze**: Erstellung einer zentralen, einheitlichen Datenbasis.
3. **Markierung besonderer Tage**: Identifikation der Kieler Woche-Tage (`KielerWoche`).
4. **Analyse-Ready**: Der bereinigte Datensatz dient als Grundlage für Analysen oder Modelle.


In [22]:
import pandas as pd

# Import Data
kiwo_data = pd.read_csv('kiwo.csv', parse_dates=['Datum'])
umsatzdaten_gekuerzt_data = pd.read_csv('umsatzdaten_gekuerzt.csv', parse_dates=['Datum'])
wetter_data = pd.read_csv('wetterdaten.csv', parse_dates=['Datum'])


# Funktion zur Datenexploration
def explore_data(df, name):
    print(f"\n{'='*50}\nExploration für {name}\n{'='*50}")
    
    # Überblick
    print(f"Form: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")
    print("Erste 5 Zeilen:")
    print(df.head())
    
    # Spaltennamen und Typen
    print("\nSpalten und Datentypen:")
    print(df.dtypes)
    
    # Fehlende Werte
    print("\nFehlende Werte pro Spalte:")
    print(df.isna().sum())
    
    # Beschreibende Statistik
    print("\nBeschreibende Statistik:")
    print(df.describe(include='all'))
    
    # Duplikate
    duplicate_count = df.duplicated().sum()
    print(f"\nAnzahl der Duplikate: {duplicate_count}")

    # Wichtige Spalten
    if 'Datum' in df.columns:
        print("\nZeitraum der Daten:")
        print(f"Min Datum: {df['Datum'].min()}, Max Datum: {df['Datum'].max()}")
    else:
        print("\nKeine 'Datum'-Spalte in diesem DataFrame.")

# Exploration der einzelnen Dateien
explore_data(kiwo_data, "Kiwo Data")
explore_data(umsatzdaten_gekuerzt_data, "Umsatzdaten Gekürzt")
explore_data(wetter_data, "Wetter Data")

# Merge umsatzdaten_gekuerzt_data und wetter_data auf der Spalte 'Datum'
merged_data = pd.merge(umsatzdaten_gekuerzt_data, wetter_data, on='Datum', how='inner')

# Merge mit kiwo_data, um die 'KielerWoche'-Information hinzuzufügen
# Missing 'KielerWoche' Werte werden mit 0 gefüllt (Standard: kein Kiwo)
final_data = pd.merge(merged_data, kiwo_data[['Datum', 'KielerWoche']], on='Datum', how='left')
final_data['KielerWoche'] = final_data['KielerWoche'].fillna(0).astype(int)

# Anzahl der NaN-Werte pro Spalte berechnen
nan_counts = final_data.isna().sum()
print("Anzahl der NaN-Werte pro Spalte:")
print(nan_counts)

# Entferne Zeilen mit NaN-Werten
clean_data = final_data.dropna()

# Speichere die Ergebnisse
final_data.to_csv('final_data_withNaN.csv', index=False)  # Mit NaN
clean_data.to_csv('final_data.csv', index=False)         # Ohne NaN

# Zeilenanzahl vor und nach dem Entfernen von NaN
difference = len(final_data) - len(clean_data)
print(f"Zeilen mit NaN entfernt: {difference}")



Exploration für Kiwo Data
Form: 72 Zeilen, 2 Spalten
Erste 5 Zeilen:
       Datum  KielerWoche
0 2012-06-16            1
1 2012-06-17            1
2 2012-06-18            1
3 2012-06-19            1
4 2012-06-20            1

Spalten und Datentypen:
Datum          datetime64[ns]
KielerWoche             int64
dtype: object

Fehlende Werte pro Spalte:
Datum          0
KielerWoche    0
dtype: int64

Beschreibende Statistik:
                     Datum  KielerWoche
count                   72         72.0
mean   2015-12-23 00:00:00          1.0
min    2012-06-16 00:00:00          1.0
25%    2014-03-24 00:00:00          1.0
50%    2015-12-23 00:00:00          1.0
75%    2017-09-22 00:00:00          1.0
max    2019-06-30 00:00:00          1.0
std                    NaN          0.0

Anzahl der Duplikate: 0

Zeitraum der Daten:
Min Datum: 2012-06-16 00:00:00, Max Datum: 2019-06-30 00:00:00

Exploration für Umsatzdaten Gekürzt
Form: 9334 Zeilen, 3 Spalten
Erste 5 Zeilen:
       Datum  Warengrup